In [ ]:

class abc(layers.Layer):               
    def __init__(self, out_channels, kernel_size=3):
        super(abc, self).__init__()
        self.conv = layers.Conv1D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x
abc1=abc(64)

In [ ]:
class se_fn(layers.Layer):
    def __init__(self,amp_ratio,out_channels):
        self.amp_ratio=amp_ratio
        self.gavgpool=layers.GlobalAveragePooling1D()
        self.dense1=Dense(out_channels * self.amp_ratio, activation='relu', kernel_initializer='glorot_uniform')
        self.dense2=Dense(out_channels, activation='relu', kernel_initializer='glorot_uniform')
        
        self.multiply=layers.Multiply()
        self.out_channels=out_channels
        super(se_fn, self).__init__()

    def call(self, input_tensor, training=False):
        shortcut=input_tensor
        x = self.gavgpool(input_tensor)
#         x = Reshape((1, self.out_channels))(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.multiply([x,shortcut])
        return x

    
se_fn_block =se_fn(16,128)

In [ ]:
class rese_2(layers.Layer):
    def __init__(self,amplify_ratio,filters):
        self.amplify_ratio=amplify_ratio
        self.filters=filters
        self.maxpool=layers.MaxPooling1D(pool_size=3)
        self.conv1 = layers.Conv1D(128, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(0), 
                                   kernel_initializer='he_uniform')

        self.dropout=layers.Dropout(0.1)
        self.bn1 = layers.BatchNormalization()
        self.add=layers.Add()
        super(rese_2, self).__init__()

    def call(self, input_tensor, training=False):
        x=xyz123(input_tensor)
        x=self.dropout(x)
        x=self.conv1(x)
        x=self.bn1(x)
        shortcut=x
        sefn=se_fn_block(x)
        x=self.add([x,sefn])
        x=tf.nn.relu(x)
        x=self.maxpool(x)
        return x

    
rese2_block =rese_2(16,128)

In [ ]:
class rese_2_new(layers.Layer):
    def __init__(self,amplify_ratio,filters):
        self.amplify_ratio=amplify_ratio
        self.filters=filters
        self.maxpool=layers.MaxPooling1D(pool_size=3)
        self.conv1 = layers.Conv1D(128, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(0), 
                                   kernel_initializer='he_uniform')

        self.dropout=layers.Dropout(0.1)
        self.bn1 = layers.BatchNormalization()
        self.add=layers.Add()
        super(rese_2_new, self).__init__()

    def call(self, input_tensor, training=False):
        x=xyz123_new(input_tensor)
        x=self.dropout(x)
        x=self.conv1(x)
        x=self.bn1(x)
        shortcut=x
        sefn=se_fn_block(x)
        x=self.add([x,sefn])
        x=tf.nn.relu(x)
        x=self.maxpool(x)
        return x

    
rese2_block_new =rese_2_new(16,128)

In [ ]:
layer1=Input(shape=(16000,8))
layer2=abc1(layer1)
layer3=MaxPool1D(pool_size=3,strides=None)(layer2)
layer4=rese2_block(layer3)
layer5=rese2_block_new(layer4)
layer6=rese2_block_new(layer5)
layer7=rese2_block_new(layer6)
layer8=GlobalMaxPool1D()(layer7)
final=Dense(units=256, activation='relu')(layer8)
layer10=BatchNormalization()(final)
layer11=Activation(activation)(layer10)    
layer12=Dropout(dropout_rate)(layer11)
out_class=Dense(units=8, activation='softmax')(layer12)
layer14=BatchNormalization()(final)
layer15=Activation(activation)(layer14)    
layer16=Dropout(dropout_rate)(layer15)
out_reg=Dense(units=2, activation='linear')(layer16)
model=tf.keras.Model(inputs=layer1,outputs=[out_class,out_reg])

model.summary()